In [28]:
import torch
device = "cuda:1" if torch.cuda.is_available() else "cpu"
num_samples = 1024
sample_size = 64

In [61]:
import utils
adj, feat, label, train_mask, val_mask, test_mask = utils.load_data("UAI", 123, device=device)

Loading uai dataset...


In [63]:
import numpy as np
def evaluate(A):
    results = {}

    # Eval using regular
    gcn = utils.get_gcn(feat.shape[1], int(label.max().item() + 1))
    utils.train_adj(gcn, feat, A, label, train_mask, 100)
    reg_acc = utils.eval_adj(gcn, adj, feat, label, test_mask)
    print(f"Reg acc: {reg_acc:.2%}")
    results[f"acc_full"] = reg_acc

    # Eval with batched
    for sample in [256, 64, 32, 16]:
        ct = int((A.shape[0] * A.shape[0]) / (np.sqrt(sample) * 1000))
        gcn = utils.get_gcn(feat.shape[1], int(label.max().item() + 1))
        utils.train(gcn, utils.get_dataloader(A, feat, label, train_mask, ct, sample, 64), 100)
        acc = utils.eval_adj(gcn, adj, feat, label, test_mask)
        print(f"S({sample}, {ct}) acc: {acc:.2%}")
        results[f"acc_{sample}"] = acc

    
    return results

In [64]:
import attacks
import importlib
importlib.reload(attacks)

budget = int(0.5 * adj.sum().item())
def get_pgd(budget, surrogate_lr=1e-2, epochs=30, attack_lr=1.0):
    return attacks.PGD_Attack(adj, feat, budget, label, train_mask, val_mask, epochs=epochs, device=device, surrogate_lr=surrogate_lr, attack_lr=attack_lr)
def get_partition_pgd(budget, k):
    return attacks.Partition_PGD(k, adj, feat, budget, label, train_mask, val_mask, device=device)
def get_sample_pgd(budget, s_size, s_ct):
    return attacks.Batch_PGD(s_size, s_ct, adj, feat, budget, label, train_mask, val_mask, device=device)



In [65]:
import export

baseargs = {
    'dataset': 'UAI',
    'method': 'clean'
}

In [69]:
results = evaluate(adj)
baseargs.update(results)
export.saveData("results.csv", baseargs)

Model training: 100%|██████████| 100/100 [00:07<00:00, 13.37it/s, loss=3.31]


Reg acc: 22.09%


Model training: 100%|██████████| 100/100 [01:44<00:00,  1.04s/it, loss=2.33]


S(256, 587) acc: 13.45%


Model training: 100%|██████████| 100/100 [01:03<00:00,  1.56it/s, loss=2.37]


S(64, 1175) acc: 7.17%


Model training: 100%|██████████| 100/100 [00:53<00:00,  1.87it/s, loss=2.14]


S(32, 1662) acc: 13.33%


Model training: 100%|██████████| 100/100 [00:29<00:00,  3.38it/s, loss=2.19]


S(16, 2351) acc: 13.73%


In [70]:
baseargs.update({'method': 'pgd-0.5'})
results = evaluate(get_pgd(int(0.5 * adj.sum().item()), epochs=30, surrogate_lr=1e-2, attack_lr=1))
baseargs.update(results)
export.saveData("results.csv", baseargs)

Model training: 100%|██████████| 100/100 [00:06<00:00, 14.29it/s, loss=2.53]


Reg acc: 21.60%


Model training: 100%|██████████| 100/100 [01:48<00:00,  1.08s/it, loss=2.29]


S(256, 587) acc: 7.17%


Model training: 100%|██████████| 100/100 [01:03<00:00,  1.58it/s, loss=1.89]


S(64, 1175) acc: 7.58%


Model training: 100%|██████████| 100/100 [00:51<00:00,  1.95it/s, loss=2.16]


S(32, 1662) acc: 7.17%


Model training: 100%|██████████| 100/100 [00:34<00:00,  2.89it/s, loss=2.07]


S(16, 2351) acc: 11.29%


In [59]:
for k in [2, 4, 16]:
    baseargs.update({'method': f"pgd_partition_{k}-0.5"})
    results = evaluate(get_partition_pgd(int(0.5 * adj.sum().item()), k))
    baseargs.update(results)
    export.saveData("results.csv", baseargs)

Model training: 100%|██████████| 100/100 [00:01<00:00, 51.45it/s, loss=1.16]


Reg acc: 56.81%


Model training: 100%|██████████| 100/100 [00:34<00:00,  2.89it/s, loss=0.11]


S(256, 278) acc: 64.28%


Model training: 100%|██████████| 100/100 [00:22<00:00,  4.52it/s, loss=0.01]


...

S(64, 556) acc: 65.52%


Model training: 100%|██████████| 100/100 [00:11<00:00,  8.35it/s, loss=0.02]


S(32, 787) acc: 64.10%


Model training: 100%|██████████| 100/100 [00:13<00:00,  7.39it/s, loss=0]  


S(16, 1113) acc: 66.35%


In [60]:
for sample in [[256, 20], [128, 30]]:
    # ct = int((adj.shape[0] * adj.shape[0]) / (np.sqrt(sample) * 1000))
    baseargs.update({'method': f"pgd_batch_{sample[0]}_{sample[1]}-0.5"})
    results = evaluate(get_sample_pgd(int(0.5 * adj.sum().item()), sample[0], sample[1]))
    baseargs.update(results)
    export.saveData("results.csv", baseargs)

Model training: 100%|██████████| 100/100 [00:01<00:00, 50.43it/s, loss=0.38]


Reg acc: 67.12%


Model training: 100%|██████████| 100/100 [00:34<00:00,  2.86it/s, loss=0]  


S(256, 278) acc: 69.79%


Model training: 100%|██████████| 100/100 [00:23<00:00,  4.34it/s, loss=0]  


...

S(64, 556) acc: 67.95%


Model training: 100%|██████████| 100/100 [00:14<00:00,  6.86it/s, loss=0]  


S(32, 787) acc: 68.25%


Model training: 100%|██████████| 100/100 [00:13<00:00,  7.48it/s, loss=0]  


S(16, 1113) acc: 66.82%


In [43]:
baseargs = {
    'dataset': 0.5,
    'k': 'random',
    's_size': 10,
    's_ct': -1,
}

{'dataset': 'Cora',
 'budget_pct': 0.5,
 'k': 'random',
 's_size': 10,
 's_ct': -1,
 'method': 'clean',
 'acc_16': 0.7419517040252686,
 'acc_32': 0.7751508951187134,
 'acc_64': 0.7615694403648376,
 'acc_128': 0.7842052578926086,
 'acc_256': 0.7655935883522034,
 'acc_2485': 0.6916499137878418}

In [32]:
import numpy as np
A = adj
for sample in [16, 32, 64, 128, 256, 512]:
    ct = int((A.shape[0] * A.shape[0]) / (np.sqrt(sample) * 1500))
    print(ct)

1029
727
514
363
257
181
